# 📁 **Montagem do Google Drive**
Monta o Google Drive para acessar arquivos de maneira persistente.

# 🛠️ **Instalação de Dependências**
Instala bibliotecas essenciais:
- **cloudscraper**: Contorno de bloqueios em requisições HTTP.
- **playwright**: Automação e execução de scripts web.
- **sofascore_wrapper**: Interface simplificada para interação com a API do Sofascore.

In [ ]:
# Montar o Google Drive
from google.colab import drive
drive.mount('/content/drive')

!pip install cloudscraper playwright sofascore_wrapper
!playwright install

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 8.2 MB/s eta 0:00:00
171.6 MiB [] 0% 0.0s171.6 MiB [] 0% 89.6s171.6 MiB [] 0% 64.5s171.6 MiB [] 0% 46.4s171.6 MiB [] 0% 33.9s171.6 MiB [] 0% 25.4s171.6 MiB [] 0% 17.7s171.6 MiB [] 1% 12.4s171.6 MiB [] 1% 8.8s171.6 MiB [] 2% 6.6s171.6 MiB [] 3% 5.2s171.6 MiB [] 4% 4.5s171.6 MiB [] 6% 3.9s171.6 MiB [] 6% 3.7s171.6 MiB [] 7% 3.5s171.6 MiB [] 7% 3.4s171.6 MiB [] 8% 3.3s171.6 MiB [] 9% 3.0s171.6 MiB [] 10% 3.1s171.6 MiB [] 11% 2.8s171.6 MiB [] 12% 2.6s171.6 MiB [] 13% 2.5s171.6 MiB [] 15% 2.3s171.6 MiB [] 16% 2.3s171.6 MiB [] 17% 2.1s171.6 MiB [] 18% 2.1s171.6 MiB [] 19% 2.0s171.6 MiB [] 20% 1.9s171.6 MiB [] 20% 2.0s171.6 MiB [] 20% 2.1s171.6 MiB [] 22% 1.9s171.6 MiB [] 24% 1.8s171.6 MiB [] 25% 1.7s171.6 MiB [] 26% 1.7s171.6 MiB [] 27% 1.6s171.6 MiB [] 29

# 📦 **Importações e Configurações**
- Importa módulos padrão do Python e bibliotecas (`json`, `pandas`, `asyncio`).
- Configura pandas para exibir conteúdos completos.
- Define a pasta de saída no Google Drive para armazenamento persistente dos datasets.

# ⚽️ **Mapeamento dos Campeonatos**
- Define dicionário contendo IDs de torneios e temporadas para consulta na API Sofascore.
- Edite conforme necessário para incluir ou excluir ligas.

# 🔗 **Funções Auxiliares para Requisições à API**

## 🌐 `fetch_json`
- Realiza chamadas HTTP para obter dados JSON da API Sofascore.
- Implementa throttling (atraso curto) para evitar bloqueios por excesso de requisições.

## 📅 `collect_events_for_champ`
- Coleta eventos (partidas) de um campeonato específico da API.
- Interage paginadamente, obtendo todas as partidas disponíveis.

## 📈 `process_events`
- Obtém estatísticas detalhadas das partidas (eventos) coletadas.
- Filtra apenas partidas finalizadas.
- Estrutura dados das equipes (casa e visitante), incluindo estatísticas relevantes.

# 🧹 **Função Auxiliar para Tratamento de Dados**

## 🔢 `extract_float`
- Converte strings percentuais para valores numéricos (float).
- Remove caracteres especiais e padroniza o formato decimal.

# 🚀 **Execução Principal**

- Inicializa sessão com a API Sofascore.
- Itera pelos campeonatos definidos, coletando e processando eventos e estatísticas das partidas.
- Salva datasets individuais em arquivos CSV por campeonato, contendo:
  - Estatísticas detalhadas das equipes (posse de bola, chutes, precisão de passes, faltas, escanteios).
  - Classificação automatizada do desempenho das equipes ("bom", "mediano", "ruim") baseada em critérios definidos:
    - **bom**: xG > 0.3, posse ≥ 55% e chutes no gol ≥ 7.
    - **mediano**: xG ≥ 0.15, posse ≥ 45% e chutes no gol ≥ 2.
    - Caso contrário, **ruim**.
- Consolida resultados de todos os campeonatos processados.
- Fecha sessão da API após conclusão.

# 🚀 **Execução Principal**

- Inicializa sessão com a API Sofascore.
- Itera pelos campeonatos definidos, coletando e processando eventos e estatísticas das partidas.
- Salva datasets individuais em arquivos CSV por campeonato, contendo:
  - Estatísticas detalhadas das equipes (posse de bola, chutes, precisão de passes, faltas, escanteios).
  - Classificação automatizada do desempenho das equipes ("bom", "mediano", "ruim") baseada em critérios definidos:
    - **bom**: xG > 0.3, posse ≥ 55% e chutes no gol ≥ 7.
    - **mediano**: xG ≥ 0.15, posse ≥ 45% e chutes no gol ≥ 2.
    - Caso contrário, **ruim**.
- Consolida resultados de todos os campeonatos processados.
- Fecha sessão da API após conclusão.

In [ ]:
import json
import pandas as pd
import os
import asyncio
import nest_asyncio

from sofascore_wrapper.api import SofascoreAPI

# Configurações pandas
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

# # Define a pasta de destino
# output_folder = os.getenv('OUTPUT_FOLDER', './dados_rag_new')
# os.makedirs(output_folder, exist_ok=True)

# Definir a pasta de destino no Drive
output_folder = "/content/drive/MyDrive/dados_rag_new/"

# Mapeamento de campeonatos para códigos de API
championships = {
    # "Brasileirão Betano 22931": {"tournament": 325, "season": 22931},
    # "Premier League 23776":   {"tournament": 17,  "season": 23776},
    # "La Liga 24127":          {"tournament": 8,   "season": 24127},
    # "Bundesliga 23538":       {"tournament": 35,  "season": 23538},
    # "Serie A 24644":          {"tournament": 23,  "season": 24644},
    # "Ligue 1 23872":          {"tournament": 34,  "season": 23872},
    # "Liga Portugal 24150":    {"tournament": 238, "season": 24150},
    # "Liga Profesional de Fútbol 37231": {"tournament": 155, "season": 37231},
    "Eredivisie 23873":       {"tournament": 37,  "season": 23873}
}

async def fetch_json(api: SofascoreAPI, endpoint: str) -> dict:
    """Busca JSON usando SofascoreAPI e faz throttling de 1s para evitar bloqueios."""
    try:
        data = await api._get(endpoint)
    except Exception as e:
        print(f"[ERROR] Falha ao buscar {endpoint}: {e}")
        return {}
    # Delay para evitar bloqueios
    await asyncio.sleep(0.1)
    return data

async def collect_events_for_champ(api: SofascoreAPI, championship: str, codes: dict) -> list:
    tournament, season = codes['tournament'], codes['season']
    events = []
    idx = 0
    while True:
        endpoint = f"/unique-tournament/{tournament}/season/{season}/events/last/{idx}"
        data = await fetch_json(api, endpoint)
        if not data.get('events'):
            break
        events.extend(data['events'])
        idx += 1
    print(f"{championship}: {len(events)} eventos coletados")
    return events

async def process_events(api: SofascoreAPI, events: list, championship: str) -> list:
    perf = []
    for evt in events:
        eid = evt['id']
        detail = await fetch_json(api, f"/event/{eid}")
        status_desc = detail.get('event', {}).get('status', {}).get('description')
        if status_desc != 'Ended':
            continue
        home = detail['event']['homeTeam']['name']
        away = detail['event']['awayTeam']['name']
        desc = f"{home} vs {away}"

        stats_data = await fetch_json(api, f"/event/{eid}/statistics")
        for grp in stats_data.get('statistics', []):
            if grp.get('period') == 'ALL':
                home_s, away_s = {}, {}
                for g in grp.get('groups', []):
                    for item in g.get('statisticsItems', []):
                        name = item['name']
                        home_s[name] = item.get('home', '0')
                        away_s[name] = item.get('away', '0')
                perf.extend([
                    {'championship': championship, 'event_id': eid, 'event': desc,
                     'team': home, 'opponent': away, 'home_team': home, 'is_home': True, 'stats': home_s},
                    {'championship': championship, 'event_id': eid, 'event': desc,
                     'team': away, 'opponent': home, 'home_team': home, 'is_home': False,'stats': away_s}
                ])
                break
    return perf

# Converte string percentual para float
def extract_float(val):
    s = str(val).replace('%', '').replace(',', '.').strip()
    try:
        return float(s)
    except:
        return 0.0

async def main():
    nest_asyncio.apply()
    api = SofascoreAPI()
    all_perf = []

    for champ, codes in championships.items():
        season = codes['season']
        print("season:", season)
        events = await collect_events_for_champ(api, champ, codes)
        perf = await process_events(api, events, champ)

        # Salvar CSV de cada campeonato
        rows = []
        for d in perf:
            st = d['stats']
            bp = extract_float(st.get('Ball possession', 0))
            ts = int(extract_float(st.get('Total shots', 0)))
            sot = int(extract_float(st.get('Shots on target', 0)))
            pp = extract_float(st.get('Passes accurate %', 0))
            xg = extract_float(st.get('Expected goals', 0))
            fouls = int(extract_float(st.get('Fouls', 0)))
            corners = int(extract_float(st.get('Corner kicks', 0)))
            label = 'ruim'
            if xg > 0.3 and bp >= 55 and sot >= 7:
                label = 'bom'
            elif xg >= 0.15 and bp >= 45 and sot >= 2:
                label = 'mediano'
            rows.append({
                'instruction': 'Classifique desempenho do time como bom, mediano ou ruim.',
                'input': f"Na partida {d['event']}, o time {d['team']} teve {bp}% posse, {ts} chutes, {sot} no gol, precisão de {pp}%, {corners} escanteios e {fouls} faltas.",
                'question': f"Com base nos dados, qual foi o desempenho do time {d['team']} na partida {d['event']} pelo campeonato {d['championship']}? Responda bom, mediano ou ruim.",
                'response': label,
                'match': d['event'],
                'championship': d['championship'],
                'home_team': d['home_team'],
                'season': season
            })

        df_champ = pd.DataFrame(rows)
        file_champ = os.path.join(output_folder, f"team_performance_{champ.replace(' ', '_')}.csv")
        df_champ.to_csv(file_champ, index=False, sep=';')
        print(f"Dataset de {champ} salvo em: {file_champ}")

        all_perf.extend(perf)

    # Fechar sessão HTTP
    await api.close()
    return all_perf

if __name__ == '__main__':
    nest_asyncio.apply()
    loop = asyncio.get_event_loop()
    perf_data = loop.run_until_complete(main())
    print(f"Total de registros coletados: {len(perf_data)}")


season: 23873
[ERROR] Falha ao buscar /unique-tournament/37/season/23873/events/last/11: Failed to fetch /unique-tournament/37/season/23873/events/last/11: 404
Eredivisie 23873: 312 eventos coletados
Dataset de Eredivisie 23873 salvo em: /content/drive/MyDrive/dados_rag_new/team_performance_Eredivisie_23873.csv
Total de registros coletados: 464
